In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import numpy as np
import math

In [2]:
sp500_url = "https://raw.githubusercontent.com/saikr789/stock-analysis-tool-1011/master/SP500companies.csv"

In [3]:
sp500 = pd.read_csv(sp500_url)
sp500.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500companies.csv", index=False)
sp500_sc = list(sp500['Security Code'])
sp500_sn = list(sp500['Security Name'])
sp500_companies = {str(sp500_sc[i]) : sp500_sn[i] for i in range(len(sp500_sc))}
# sp500_companies

In [4]:
def calculate_profit(data, end_date, days_to_search, investment, final_df, final_df1, sec_num):
    start_date = end_date - timedelta(days = days_to_search)
    print(start_date, end_date)
    mask = (data["date"] >= start_date) & (data["date"] <= end_date)
    data = data.loc[mask]
    data = data.reset_index(drop=True)
    inv = investment
    shares = 0
    i = 0
    inv_status = False
    ext_status = False
    first_inv = 0
    while (i <= data.shape[0] - 1):
#         print(data.loc[i, "date"])
        if (not inv_status):
            if (data.loc[i, "invest"]):
                shares = inv // data.loc[i, "close"]
                first_inv = inv
                inv = 0
                inv_status = True
                ext_status = False
                row = {}
                row["Date"] = data.loc[i, "date"]
                row["Close Price"] = data.loc[i, "close"]
                row["Invest/Exit"] = "INVEST"
                row["Investment"] = inv
                row["Shares"] = shares
                row["Returns %"] = 0
                final_df = final_df.append(row, ignore_index=True)
        else:
            if (data.loc[i, "exit"]):
                inv = shares * data.loc[i, "close"]
                shares = 0
                ext_status = True
                inv_status = False
                avg = (inv - first_inv)/ first_inv
                row = {}
                row["Date"] = data.loc[i, "date"]
                row["Close Price"] = data.loc[i, "close"]
                row["Invest/Exit"] = "EXIT"
                row["Investment"] = inv
                row["Shares"] = shares
                row["Returns %"] = avg
                final_df = final_df.append(row, ignore_index=True)
        i += 1
        
    
    if ((not inv_status) and (not ext_status)):
        row = {}
        row["Date"] = data.loc[data.shape[0] - 1, "date"]
        row["Close Price"] = data.loc[data.shape[0] - 1, "close"]
        row["Invest/Exit"] = "-"
        row["Investment"] = 0
        row["Shares"] = 0
        row["Returns %"] = 0
        final_df = final_df.append(row, ignore_index=True)
    
    if (inv_status and (not ext_status)):
        inv = shares * data.loc[data.shape[0] - 1, "close"]
        avg = (inv - first_inv)/ first_inv
        shares = 0
        row = {}
        row["Date"] = data.loc[data.shape[0] - 1, "date"]
        row["Close Price"] = data.loc[data.shape[0] - 1, "close"]
        row["Invest/Exit"] = "-"
        row["Investment"] = inv
        row["Shares"] = shares
        row["Returns %"] = avg
        final_df = final_df.append(row, ignore_index=True)
    
    total_avg = final_df['Returns %'].sum() / final_df[final_df["Invest/Exit"].isin(["EXIT", "-"])].shape[0]
    company = sp500_companies[sec_num]
    r = {}
    r["Company"] = company
    r["Security Id"] = sec_num
    r["Average Returns %"] = total_avg
#     r["Days"] = days_to_search
    final_df1 = final_df1.append(r, ignore_index=True)
    
    return inv, shares, final_df, final_df1
        
            

In [5]:
# columns = ["Date", "Close Price", "Invest/Exit", "Investment", "Shares"]
# fd_df = pd.DataFrame(columns = columns)
# fd_df

In [6]:
# columns1 = ["Company", "Average Returns"]
# fd_df1 = pd.DataFrame(columns = columns1)
# fd_df1

In [7]:
# df1 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_input_files\\simulation\\500002.csv")
# # print(df1.shape)
# df1["date"] = pd.to_datetime(df1["date"])
# ed_date = df1.loc[df1.shape[0] - 1, "date"]
# # days = df1.shape[0]
# days = 60
# inst = 100000
# sn = "500002"
# investment, shares, final_df , fd_df1 = calculate_profit(df1, ed_date, days, inst, fd_df, fd_df1, sn)
# print(investment, shares)
# print(final_df)
# print(fd_df1)
# final_df.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\NextUBLB_Results\\" + name + "_Results.csv", index = False)

In [8]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

In [9]:
# columns1 = ["Company", "Average Returns", "Days"]
# fd_df1 = pd.DataFrame(columns = columns1)
# fd_df1

In [10]:
%%time


def get_simulation(days):
#     columns1 = ["Company", "Average Returns", "Days"]
    columns1 = ["Company", "Security Id", "Average Returns %"]
    fd_df1 = pd.DataFrame(columns = columns1)
    
    for filename in os.listdir(os.path.join(path,"Data/SP500_simulation_input_files/simulation")):
        columns = ["Date", "Close Price", "Invest/Exit", "Investment", "Shares"]
        fd_df = pd.DataFrame(columns = columns)
        print("-------------------------------------------------------")
        name = filename[ : 6]
        print(name)
        df1 = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_input_files\\simulation\\" + filename)
        df1["date"] = pd.to_datetime(df1["date"])
        ed_date = df1.loc[df1.shape[0] - 1, "date"]
        inst = 100000
        investment, shares, final_df, fd_df1 = calculate_profit(df1, ed_date, days, inst, fd_df, fd_df1, name)
#         print(investment, shares)
#         print(final_df)
#         print(fd_df1)
        final_df.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\" + name + "_" + str(days) + "days.csv", index = False)
        print("-------------------------------------------------------")

    fd_df1 = fd_df1.sort_values(by = ['Average Returns %'], ascending = [False])
    print(fd_df1)
    fd_df1.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\SP500_simulation_results\\top_" + str(days) + "days_Results.csv", index = False)



Wall time: 0 ns


In [11]:
%%time

days = [30, 60, 90, 180, 270, 360, 540, 720, 900, 1080]

for d in days:
    print(d)
    print("*****************************************************************")
    get_simulation(d)
    print("*****************************************************************")

30
*****************************************************************
-------------------------------------------------------
500002
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500003
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500008
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500010
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500020
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500027
2021-03-17 00:00:00 2021-04-16 00:00:00
---------------------------------

-------------------------------------------------------
-------------------------------------------------------
500251
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500252
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500253
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500257
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500260
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500265
2021-03-17 00:00:00 2021-04-16 00:00:00
----------------------------------------------

-------------------------------------------------------
-------------------------------------------------------
500660
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500670
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500674
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500680
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500690
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
500696
2021-03-17 00:00:00 2021-04-16 00:00:00
----------------------------------------------

-------------------------------------------------------
-------------------------------------------------------
508989
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
509243
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
509480
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
509488
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
509557
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
509631
2021-03-17 00:00:00 2021-04-16 00:00:00
----------------------------------------------

-------------------------------------------------------
524164
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
524200
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
524208
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
524230
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
524348
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
524494
2021-03-17 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
----------------------------------------------

...

-------------------------------------------------------
-------------------------------------------------------
532162
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532174
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532175
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532178
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532181
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532187
2018-05-02 00:00:00 2021-04-16 00:00:00
----------------------------------------------

-------------------------------------------------------
-------------------------------------------------------
532555
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532612
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532622
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532636
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532638
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532644
2018-05-02 00:00:00 2021-04-16 00:00:00
----------------------------------------------

-------------------------------------------------------
532922
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532926
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532927
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532929
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532942
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
532947
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
----------------------------------------------

-------------------------------------------------------
-------------------------------------------------------
539448
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
539450
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
539523
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
539524
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
539542
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
539551
2018-05-02 00:00:00 2021-04-16 00:00:00
----------------------------------------------

-------------------------------------------------------
541450
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
541540
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
541556
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
541557
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
541578
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
-------------------------------------------------------
541700
2018-05-02 00:00:00 2021-04-16 00:00:00
-------------------------------------------------------
----------------------------------------------